In [1]:
import numpy as np
import tensorflow as tf

/Users/brianregan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [119]:
n_boat = 15
n_anchor = 9
img_size = 5
batch_size = 10

In [168]:
# Simpler Case 
# Array Creation
# main array is a [batch_size, h, w, n_boat, 4] shaped array where values are repeated along the last dimension.
# in the first slice of the n_boat dimension each array is batch_index*[[0,1,2,3,4], [1,2,3,4,5], ...] while the second
# slice in the n_boat dimension is batch_index*[[[1,2,3,4,5], [2,3,4,5,6]] ...] (ie. shifted up one).
np.random.seed(seed=0)

batch_arrs = []
for b in range(batch_size):
    arrs = []
    for i in range(n_boat):
        arrs.append(np.arange(start=i, stop=i +img_size*img_size).reshape((img_size, img_size)))
    
    batch_array = np.stack(arrs, -1)
    batch_array = np.repeat(np.expand_dims(batch_array, -1), 4, -1)
    #main_array = np.repeat(np.expand_dims(main_array, 0), batch_size, 0)
    batch_arrs.append(batch_array*b)

main_array = np.stack(batch_arrs, 0)
    

index_array = np.random.randint(n_boat, size=(img_size, img_size))
index_array = np.repeat(np.expand_dims(index_array, 0), batch_size, 0)

In [189]:
# [batch_size, h, w, n_boat, 4]
main_array.shape

(10, 5, 5, 15, 4)

In [191]:
# [batch_size, h, w]
index_array.shape

(10, 5, 5)

In [183]:
main_array[1,:,:,2, 0]

array([[ 2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21],
       [22, 23, 24, 25, 26]])

In [194]:
index_array[3,:,:]

array([[12,  5,  0,  3, 11],
       [ 3,  7,  9,  3,  5],
       [ 2,  4,  7,  6,  8],
       [ 8, 12, 10,  1,  6],
       [ 7,  7, 14,  8,  1]])

In [198]:
# See https://stackoverflow.com/questions/28980345/numpy-index-array-by-array
out = main_array[np.arange(batch_size)[:,np.newaxis, np.newaxis], 
                 np.arange(img_size)[np.newaxis, :, np.newaxis], 
                 np.arange(img_size)[np.newaxis, np.newaxis, :], 
                 index_array, :]



In [213]:
def select_with_matrix(arr, indexer):
    '''
    
    '''
    
    batch_size, h, w, depth, vals = arr.shape
    out = arr[np.arange(batch_size)[:,np.newaxis, np.newaxis], 
              np.arange(h)[np.newaxis, :, np.newaxis], 
              np.arange(w)[np.newaxis, np.newaxis, :], 
              indexer, :]
    return out

In [215]:
out2 = select_with_matrix(main_array, index_array)

In [217]:
out2 - out

array([[[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]],


       [[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 

In [153]:
out.shape

(10, 5, 5, 4)

In [208]:
out[1, :,:,0]

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])